In [1]:
import os
from PyPDF2 import PdfReader
import docx
import re
import pytesseract
from PIL import Image
import io
import fitz
from docx.document import Document
from docx.oxml.shape import CT_Picture
from docx.parts.image import ImagePart

def clean_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace('\u00a0', ' ')
    text = text.replace('\n', ' ').replace('\r', '')
    return text

def perform_ocr(image):
    return pytesseract.image_to_string(image)

def extract_pdf_text(file_path):
    text = ''
    with fitz.open(file_path) as pdf:
        for page in pdf:
            text += page.get_text()
            
            for img in page.get_images():
                xref = img[0]
                base_image = pdf.extract_image(xref)
                image_bytes = base_image["image"]
                image = Image.open(io.BytesIO(image_bytes))
                text += ' ' + perform_ocr(image)
    
    return clean_text(text)

def extract_docx_text(file_path):
    doc = docx.Document(file_path)
    text = '\n'.join([paragraph.text for paragraph in doc.paragraphs])
    
    for rel in doc.part.rels.values():
        if isinstance(rel._target, ImagePart):
            image_bytes = rel._target.blob
            image = Image.open(io.BytesIO(image_bytes))
            text += ' ' + perform_ocr(image)
    
    return clean_text(text)

def extract_txt_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return clean_text(file.read())

def extract_image_text(file_path):
    image = Image.open(file_path)
    text = perform_ocr(image)
    return clean_text(text)

def upload_and_parse_document(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.pdf':
        return extract_pdf_text(file_path)
    elif file_extension in ['.docx', '.doc']:
        return extract_docx_text(file_path)
    elif file_extension == '.txt':
        return extract_txt_text(file_path)
    elif file_extension in ['.png', '.jpg', '.jpeg', '.tiff', '.bmp']:
        return extract_image_text(file_path)
    else:
        raise ValueError('Unsupported file type')

In [2]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\suhrp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def tokenize_text(text):
    return word_tokenize(text.lower())

def vectorize_text(text, word2vec_model):
    tokens = tokenize_text(text)
    word_vectors = [word2vec_model.wv[word] for word in tokens if word in word2vec_model.wv]
    if not word_vectors:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(word_vectors, axis=0)


def process_document_vector(file_path):
    text = upload_and_parse_document(file_path)
    
    tokenized_text = [tokenize_text(text)]
    
    word2vec_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)
    
    text_vector = vectorize_text(text, word2vec_model)
    
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([text])
    
    tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
    tfidf_word_vectors = []
    
    for word in tfidf_feature_names:
        if word in word2vec_model.wv:
            word_vector = word2vec_model.wv[word]
            tfidf_score = tfidf_matrix[0, tfidf_vectorizer.vocabulary_[word]]
            tfidf_word_vectors.append(word_vector * tfidf_score)
    
    if tfidf_word_vectors:
        final_vector = np.mean(tfidf_word_vectors, axis=0)
    else:
        final_vector = np.zeros(word2vec_model.vector_size)
    
    return final_vector

In [4]:
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

def get_vector_db(file_path):
    text = upload_and_parse_document("Pre-Implementation Design Report.pdf")
    embeddings = GPT4AllEmbeddings()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=50,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    persist_directory = "./chroma_db"
    vectordb = Chroma.from_texts(
        texts=chunks,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    return vectordb

c:\Users\suhrp\Downloads\document-analysis-thing\final-task\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in GPT4AllEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [5]:
def document_pipeline(file_path):
    text = upload_and_parse_document(file_path)
    final_vec = process_document_vector(file_path)
    vector_db = get_vector_db(file_path)
    return text, final_vec, vector_db


In [6]:
text_representation, final_vector, vector_chroma = document_pipeline("Pre-Implementation Design Report.pdf")

In [7]:
print(text_representation)
print(final_vector)
print(vector_chroma)

Task Overview Develop a prototype for an advanced document analysis system using transformer-based models, incorporating real-time annotation capabilities. This task is designed to be completed in 8-10 hours and should showcase your team's ability to work with state-of-the-art NLP models, handle data processing, and create an interactive user interface. Project Goals 1. Implement a document ingestion and preprocessing pipeline ● Take pdf and store in mongo db ● Upload newly annotated documents to mongo db upon finishing annotation, for the sake of version history and referencing ● Create one pipeline function which will accept all the text in the initial file and then repeatedly accept new text as real time annotation happens ○ Create a function that accepts a file path and set up conditionals to check what the file extension is. ■ Use PyPDF2’s PdfReader module to iterate through pdf documents and use the extract text built in function ■ Use docx module to handle situations where the d